# Multiclass Multiple Kernel Learning using SHOGUN

Le but de ce Notebook est d'expliquer, étapes par étapes, ce que réalise ce code, inspiré de l'exemple fourni par Shogun.

## Principe du Multiple Kernel Learning

Un algorithme de Multiple Kernel Learning (MKL) a pour but de combiner plusieurs kernels, de façon optimale, en vue d'optimiser la prédiction du classifieur.

Avoir recours au MKL peut avoir deux intérêts :
- Proposer plusieurs kernels au "Learner" pour un même jeu de données, afin que celui-ci puisse choisir le ou les kernels adapté(s)
- Avoir recours à plusieurs données hétérogènes auxquelles sont associées un ou plusieurs kernels, et pouvoir les combiner pour bénéficier de ces différentes sources d'information. 

Sur Shogun, le "Learner" utilisé est la SVM. 
Pour la partie MKL, Shogun pose un problème d'optimisation dans lequel il va tenter de minimiser la somme de deux termes : un terme d'erreur et un terme de complexité. Il s'agit de la méthode de minimisation des risques structurels. 
Il recherche ainsi à résoudre un problème en formulation SILP (Semi-Infinite Linear Programming), et suit pour cela une méthode en deux étapes : 
- dans la première, il optimise le vecteur de poids des kernels
- dans la deuxième, il trouve les paramètres de la SVM 

Ces deux itérations s'enchaînent jusqu'à convergence. 

##### La section suivante explicite plus en détail le problème d'optimisation résolu par Shogun. Il s'agit d'un développement théorique. Passez à la section suivante pour l'explication du code. 

Pour mieux comprendre le fonctionnement de cette librairie, on peut s'interesser à la façon dont le problème de classification est résolu. 

Il s'agit en fait de reformuler le problème sous une forme SILP, c'est-à-dire que la fonction objectif et les contraintes seront linéaires, mais qu'il y a une infinité de contraintes.

Ces contraintes sont relatives aux valeurs alpha des multiplicateurs lagrangiens du problème posé par la SVM. Ces valeurs peuvent varier sur le segment [0 ; C] (où C représente le coût de la classification). Il y a donc une infinité de possibilités. 

Pour résoudre le problème, l'algorithme va fonctionner en deux étapes.
- Il va tout d'abord prendre un lot fini de contraintes et trouver la solution optimale du sous-problème ainsi posé.
- Il entre alors dans la deuxième étape, où il va générer, à partir de la solution de la première étape, la contrainte la plus sévère parmi l'ensemble des contraintes inexploitées. 
- Cette contrainte est alors ajoutée au lot de contraintes de la première étape.
    - Si la contrainte est satisfaite, alors cela veut dire que la solution est optimale, et l'algorithme s'arrête
    - Si la contrainte n'est pas satisfaite, alors on trouve une nouvelle solution au sous-problème et on reboucle. 
    
Le critère d'arrêt de l'algorithme est donc lié à la valeur de la violation de la contrainte la plus sévère. On définit ainsi un seuil mkl_epsilon, tel que, si la valeur de la violation est inférieure à ce seuil, alors on considère que la solution est optimale. 

Voyons maintenant en quoi consiste l'algorithme.

In [1]:
#import sys
#import os
#shogun_path="/usr/local/lib/python2.7/site-packages/"
#sys.path.append(shogun_path)
#print(sys.path)
#from modshogun import *

Anciennement, on utilisait les instructions ci-dessus pour importer les éléments nécessaires. 

Il est également possible de faire comme suit (maintenant que l'utilisation n'est plus locale sur ozzy). 

In [2]:
import sys
import os
from modshogun import *

In [3]:
%pylab inline
%matplotlib inline
from scipy.io import loadmat, savemat
from os       import path, sep
import time

/home/global/anaconda2/lib/python2.7/site-packages/PIL/Image.py:77: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL: Core Version: 4.1.1Pillow Version:  4.2.1
  warnings.warn(str(v), RuntimeWarning)
/home/global/anaconda2/lib/python2.7/site-packages/PIL/Image.py:77: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL: Core Version: 4.1.1Pillow Version:  4.2.1
  warnings.warn(str(v), RuntimeWarning)


Populating the interactive namespace from numpy and matplotlib


/home/global/anaconda2/lib/python2.7/site-packages/PIL/Image.py:77: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL: Core Version: 4.1.1Pillow Version:  4.2.1
  warnings.warn(str(v), RuntimeWarning)


Après avoir importé les éléments dont nous avons besoin pour faire fonctionner l'algorithme, nous définissons nos données.

In [4]:
mat  = loadmat('/NAS/dumbo/protocoles/CogPhenoPark/data/cogphenoparkCli3.mat')
mat2  = loadmat('/NAS/dumbo/protocoles/CogPhenoPark/data/cogphenopark_New.mat')
mat3 = loadmat('/NAS/dumbo/protocoles/CogPhenoPark/data/cogphenopark_anova.mat') 
mat4 = loadmat('/home/nrossignol/Documents/Score_Cli_normalised.mat')
mat5 = loadmat('/home/nrossignol/Documents/Connectome_normalised.mat')
mat6 = loadmat('/home/nrossignol/Documents/MATLAB/new_dataCliCopy.mat')
mat7 = loadmat('/home/nrossignol/Documents/MATLAB/new_conn_norm.mat')
mat8 = loadmat('/home/nrossignol/Documents/MATLAB/new_label.mat')

mat9 = loadmat('/home/nrossignol/Documents/MATLAB/score_cli1.mat')
mat10 = loadmat('/home/nrossignol/Documents/MATLAB/score_cli2.mat')
mat11 = loadmat('/home/nrossignol/Documents/MATLAB/score_cli3.mat')

Nous créons ensuite deux éléments : 
- une matrice X qui contient l'ensemble des features de nos patients, de dimension nb_features x nb_individus
- un vecteur Y qui contient les classes (ou labels) de chaque individu, i.e un vecteur ligne de dimension nb_individus

Précisons que si nous avons plus de 2 classes, il faut attribuer à chacune d'entre elle un nombre entier, en partant de 0. (0, 1, 2, ...)

Par ailleurs, si on travaille sur plusieurs jeux de données, alors on définit plusieurs vecteurs X, à raison de un pour chaque jeu de features. 

In [5]:
#Xall1 = mat['dataCli'] #z-scores cliniques
#Xall2 = mat2['newData'] #connectomes
#Xall2 = mat3['data'] #connectomes_anova

Xall1 = mat4['dataCliCopy'] #z-scores cliniques normalisés
Xall2 = mat5['conn_norm'] #connectomes normalisés

#Xall1 = mat6['new_dataCliCopy'] #z-scores cliniques normalisés classes 1, 3 et 4
#Xall2 = mat7['new_conn_norm'] #connectomes normalisés classes 1, 3 et 4

#Xall1 = mat9['score_cli1'] #z-scores cliniques normalisés
#Xall2 = mat10['score_cli2'] #connectomes normalisés
#Xall3 = mat11['score_cli3'] #connectomes normalisés

Yall = array(mat['label'].squeeze(), dtype=double)

print Xall1.shape
print Xall2.shape
#print Xall3.shape

Yall = Yall - 1
#on fait partir nos classes de 0

print Yall

(3, 112)
(13366, 112)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.
  4.  4.  4.  4.]


## Approche générale

On explique ici le principe de l'algorithme.
Celui-ci étant amené à fonctionner en Leave-one-out, on se fixe un i pour étudier ce que fait l'algorithme lors d'une itération. 

On définit une liste d'indices "ind" qui fait la même taille que notre vecteur des labels Y. Si la longueur de Y est n, alors ind sera la liste des entiers de 0 à n-1 [0, ..., n-1]. Cela ne présente pas de problème pour python, le premier élément d'une liste est l'élément d'indice 0. On ne perd donc pas d'individu avec cette démarche.

De cette liste, on va extraire le i-ème élément, qui sera l'indice de notre élément de test.

In [6]:
i = 40 
ind = range (len(Yall))
del ind[i]

On définit maintenant la matrice de nos données d'entraînement, Xtrain, à raison de 1 pour chaque jeu de features. 
Si le but est de réaliser du Leave-one-out, il suffit de reprendre l'ensemble de nos lignes (qui correspondent à nos features), ainsi que toutes les colonnes de la liste ind (c'est à dire toutes sauf la i-ème). 

On définit également notre vecteur label Ytrain associé à nos données d'entraînement de la même façon.

Afin de reconnaître les différentes classes contenues dans Ytrain, on utlise la fonction MulticlassLabels().
Cette fonction n'est utile que dans le cas d'un nombre de classes supérieur à 2. Il faut faire appel à une autre fonction pour la classification binaire.

De même, on extrait les features de nos données contenues dans Xtrain à l'aide de la fonction Realfeatures(). Cette fonction est la même, que l'on soit en binaire ou en multi-classes. 

Enfin, on refait les mêmes opérations pour nos données de test. 

In [7]:
Xtrain1 = Xall1[:,ind]
Xtrain2 = Xall2[:,ind]
#Xtrain3 = Xall3[:,ind]

Ytrain = Yall[ind]

labels = MulticlassLabels(Ytrain)

feats1  = RealFeatures(Xtrain1)
feats2  = RealFeatures(Xtrain2)
#feats3  = RealFeatures(Xtrain3)

Xrem1 = Xall1[:, i:i+1]
Xrem2 = Xall2[:, i:i+1]
#Xrem3 = Xall3[:, i:i+1]

Yrem = Yall[i:i+1]

labels_rem = MulticlassLabels(Yrem)

feats_rem1 = RealFeatures(Xrem1)
feats_rem2 = RealFeatures(Xrem2)
#feats_rem3 = RealFeatures(Xrem3)

On définit ici de nouvelles variables.

La fonction CombinedFeatures() permet de créer une liste de l'ensemble des features associées à chacun de nos jeux de données, tout en gardant un pointeur vers le kernel auquel ces features sont associées. 
Par exemple, si j'ai un total de deux kernels associés chacun à un jeu de features, la fonction CombinedFeature() contiendra toutes les features, divisées en 2 sous-lots pointant chacun vers leur kernel respectif. 

Parallèlement, la fonction CombinedKernel() contiendra tous les kernels que l'on souhaite utiliser dans notre classification. 

Précision : 

Si l'on souhaite utiliser plusieurs kernels sur un même jeu de données, il y a 2 possibilités :
- la fonction CombinedFeatures() peut prendre autant de sous-lots de features qu'il y a de kernels (tous les sous-lots seront alors identiques)
- on peut ne donner qu'un unique lot à la fonction CombinedFeatures, qui sera par défaut associé à l'ensemble des kernels contenus dans CombinedKernels()

In [8]:
feats_train = CombinedFeatures()
feats_test = CombinedFeatures()
combined_kernel = CombinedKernel()

Shogun dispose de nombreux types de kernels. Les plus fréquemment utilisés restent cependant les kernels Gaussiens, Polynomiaux et Linéaires. 

- Pour le kernel Gaussien, il n'y a qu'un hyperparamètre à fixer : la largeur de bande.
- Pour le kernel Polynomial, il y a deux hyperparamètres : le premier correspond au coefficient dans l'expression du kernel polynomial, et le deuxième au degré. 

Il est important de préciser, à chaque fois qu'un kernel est créé, le jeu de features auquel il est rattaché, et d'intégrer celui-ci au CombinedFeatures(). 
De même, le kernel doit être incorporé au CombinedKernel(). 

Quelques informations sur les hyperparamètres :

Plus on diminue la largeur de bande d'un kernel Gaussien, plus on se rapproche d'un comportement linéaire.
Plus on l'augmente, plus on risque l'overfeating sur nos données d'entraînement.

Pour visualiser ce comportement, consulter : https://remi.flamary.com/demos/svmreg.fr.html

In [9]:
#append gaussian kernel

subkernel = GaussianKernel(0.1)        
feats_train.append_feature_obj(feats1)
feats_test.append_feature_obj(feats_rem1)
combined_kernel.append_kernel(subkernel)

#append PolyKernel

subkernel = PolyKernel(5,3)            
feats_train.append_feature_obj(feats1)
feats_test.append_feature_obj(feats_rem1)
combined_kernel.append_kernel(subkernel)
    
#append Linear Kernel

subkernel = LinearKernel()            
feats_train.append_feature_obj(feats1)
feats_test.append_feature_obj(feats_rem1)
combined_kernel.append_kernel(subkernel)


True

### Phase d'entraînement

La fonction init() permet de lancer l'algorithme. Pour cela, on lui précise deux bases.
La première base correspond à la base d'entraînement. La deuxième dépend de ce que l'on souhaite faire :
- si on veut réaliser l'entraînement, on précise à nouveau la base d'entraînement
- si on veut réaliser le test, on précise la base de test.

La fonction MKLMulticlass(C, K, l) prends en arguments le coût C de la classification, le CombinedKernel() et l'ensemble des labels. 

C'est à l'utilisateur de fixer la valeur de C, qui peut avoir une influence sur la qualité de la classification.

Le paramètre mkl_epsilon renvoit au critère d'arrêt du problème d'optimisation. Plus celui-ci est faible, plus on se rapproche de la solution optimale du problème d'optimisation. Mais cela augmente également le temps d'éxécution de l'algorithme.

Enfin, mkl_norm renvoie à la contrainte que l'on impose sur la somme des poids des kernels. En effet, en norme 1, on impose que la somme des poids soit égale à un, mais on pourrait tout aussi bien imposer que la somme des carrés des poids soit égale à 1, et on utiliserait pour cela la norme 2 (cela permettrait d'avoir une pondération dense des kernels, par opposition à la solution sparse que renvoie la normalisation L1).

Enfin, l'entraînement est réalisé avec la fonction train()

Précisions sur l'hyperparamètre C :

C est un hyperparamètre qui influence à la quantité d'erreurs de classifications que s'autorise le classifieur. Plus C est grand, plus on autorise d'erreurs. Toutefois, le problème d'optimisation sur lequel s'appuie la classification vise à minimiser la valeur de ces erreurs. Par conséquent plus C est grand, plus il y a d'erreurs à minimiser et plus la complexité de l'algorithme augmente. Mais en contrepartie, si notre problème est non séparable, alors il est nécessaire de s'autoriser un minimum d'erreurs de classification afin d'être capable de trouver un hyperplan séparateur. 


In [10]:
combined_kernel.init(feats_train, feats_train)
mkl = MKLMulticlass(1.2, combined_kernel, labels)

#mkl.set_mkl_epsilon(1e-5)
mkl.set_mkl_norm(1)

#Pour la classification binaire
    
#mkl = MKLClassification()
#mkl.set_C(1, 1)
#mkl.set_kernel(combined_kernel)
#mkl.set_labels(labels)
    
mkl.train()

True

### Phase de test

On utilise maintenant la fonction init() pour réaliser notre test sur l'élément d'indice i. 

La fonction apply() permet de faire fonctionner l'algorithme pour nos données de test. On stocke la prédiction dans une variable 'out'.

On peut remonter à la valeur de l'exactitude en comparant notre variable de sortie 'out' avec les valeurs de prédiction attendues stockées dans label_rem. pour cela on utilise les fonctions MulticlassAccuracy() et evaluate()

Il peut également être interessant de visulatiser les poids attribués à chacun de nos kernels, via la fonction get_subkernel_weights().

In [11]:
combined_kernel.init(feats_train, feats_test)     

out =  mkl.apply()
evaluator = MulticlassAccuracy()

acc = evaluator.evaluate(out, labels_rem)
    
w = combined_kernel.get_subkernel_weights()
    
print w
    
print "Accuracy globale = %2.2f%%" % (100*acc)

[ 0.63447405  0.          0.36552595]
Accuracy globale = 100.00%


### Méthode de Leave-one-out

Afin de réaliser du Leave-one-out, nous avons construit la méthode suivante :

- A chaque itération de la boucle for, on retire le i-ème élément de ind, qui constituera notre élément de test. 

-  le vecteur de poids peut être amené à varier (l'optimisation est refaite à chaque fois). On peut donc volontairement le visualiser à chaque fois, pour constater son évolution entre chaque itération.

- En fonction du résultat de la prédiction (0 si mal classé, 1 si bien classé), on augmente la valeur de accuracy_globale, qui, une fois rapportée à la taille de l'effectif, permettra d'évaluer l'exactitude sur l'ensemble des prédictions.

- On peut également faire la même chose pour chaque classe individuelle. Il suffit pour cela de connaître leur effectif. 

- On peut aussi remonter à la prédiction que fait notre algorithme grâce à la fonction get_labels().

Afin d'évaluer la durée d'éxécution de toute cette étape, il peut être bon de relever le temps affiché par l'horloge au début (tic) et à la fin (toc). 

In [12]:
accuracy = []
prediction = []
accuracy_globale = 0

tic = time.time()

#LOO

for i in range(len(Yall)) :

    ind = range (len(Yall))
    del ind[i]

    Xtrain1 = Xall1[:,ind]
    Xtrain2 = Xall2[:,ind]
    #Xtrain3 = Xall3[:,ind]

    Ytrain = Yall[ind]

    labels = MulticlassLabels(Ytrain)

    feats1  = RealFeatures(Xtrain1)
    feats2  = RealFeatures(Xtrain2)
    #feats3  = RealFeatures(Xtrain3)

    Xrem1 = Xall1[:, i:i+1]
    Xrem2 = Xall2[:, i:i+1]
    #Xrem3 = Xall3[:, i:i+1]

    Yrem = Yall[i:i+1]

    labels_rem = MulticlassLabels(Yrem)

    feats_rem1 = RealFeatures(Xrem1)
    feats_rem2 = RealFeatures(Xrem2)
    #feats_rem3 = RealFeatures(Xrem3)
    
# MKL training and output

    feats_train = CombinedFeatures()
    feats_test = CombinedFeatures()
    combined_kernel = CombinedKernel()


##### Pour le premier jeu de features ######    
       
#append gaussian kernel 

    subkernel = GaussianKernel(1)        
    feats_train.append_feature_obj(feats1)
    feats_test.append_feature_obj(feats_rem1)
    combined_kernel.append_kernel(subkernel)
    
#append gaussian kernel

    subkernel = GaussianKernel(0.1)        
    feats_train.append_feature_obj(feats1)
    feats_test.append_feature_obj(feats_rem1)
    combined_kernel.append_kernel(subkernel)
    

#append PolyKernel

    #subkernel = PolyKernel(10,3)            
    #feats_train.append_feature_obj(feats1)
    #feats_test.append_feature_obj(feats_rem1)
    #combined_kernel.append_kernel(subkernel)
    
#append Linear Kernel

    #subkernel = LinearKernel()            
    #feats_train.append_feature_obj(feats1)
    #feats_test.append_feature_obj(feats_rem1)
    #combined_kernel.append_kernel(subkernel)


##### Pour le deuxième jeu de features ######    
    
#append gaussian kernel

    subkernel = GaussianKernel(0.1)        
    feats_train.append_feature_obj(feats2)
    feats_test.append_feature_obj(feats_rem2)
    combined_kernel.append_kernel(subkernel) 

#append PolyKernel

    #subkernel = PolyKernel(10,3)            
    #feats_train.append_feature_obj(feats2)
    #feats_test.append_feature_obj(feats_rem2)
    #combined_kernel.append_kernel(subkernel)
    
#append Linear Kernel

    #subkernel = LinearKernel()            
    #feats_train.append_feature_obj(feats2)
    #feats_test.append_feature_obj(feats_rem2)
    #combined_kernel.append_kernel(subkernel)
    
##### Pour le troisième jeu de features ######    
    
#append gaussian kernel

    #subkernel = GaussianKernel(1)        
    #feats_train.append_feature_obj(feats3)
    #feats_test.append_feature_obj(feats_rem3)
    #combined_kernel.append_kernel(subkernel) 


    combined_kernel.init(feats_train, feats_train)
    
    mkl = MKLMulticlass(10, combined_kernel, labels)

    #mkl.set_epsilon(1e-3)
    mkl.set_mkl_epsilon(1e-3)
    mkl.set_mkl_norm(1)
    
    mkl.train()

#MKL Test
    
    combined_kernel.init(feats_train, feats_test)     

    out =  mkl.apply()
    evaluator = MulticlassAccuracy()

    acc = evaluator.evaluate(out, labels_rem)
    accuracy.append(acc)
    accuracy_globale = accuracy_globale + acc
    
    w = combined_kernel.get_subkernel_weights()
    
    prediction.append(out.get_labels().tolist())
    result = array(prediction).T.tolist()[0]
    #z = out.get_values()
    
    print w
    
print "Accuracy globale = %2.2f%%" % (100*accuracy_globale/len(Yall))

toc = time.time()

[ 0.06189442  0.20620876  0.73189681]
[ 0.06198848  0.20927068  0.72874084]
[ 0.06209331  0.20710758  0.7307991 ]
[ 0.0621794   0.21224113  0.72557947]
[ 0.06214283  0.20904841  0.72880876]
[ 0.06204713  0.20778195  0.73017092]
[ 0.06219707  0.20522413  0.73257879]
[ 0.06198714  0.20893339  0.72907947]
[ 0.06208561  0.2071782   0.73073619]
[ 0.06194841  0.20815456  0.72989702]
[ 0.06249116  0.20703896  0.73046988]
[ 0.06227355  0.2151269   0.72259955]
[ 0.06224511  0.21227528  0.72547961]
[ 0.06201072  0.2078261   0.73016318]
[ 0.06197178  0.20853127  0.72949695]
[ 0.06232975  0.21061082  0.72705944]
[ 0.06189851  0.20612011  0.73198138]
[ 0.06214449  0.20884066  0.72901484]
[ 0.06203752  0.20834784  0.72961464]
[ 0.0621315   0.20915769  0.72871081]
[ 0.06247242  0.20666038  0.7308672 ]
[ 0.06206821  0.20952895  0.72840284]
[ 0.06234245  0.21299917  0.72465838]
[ 0.0621833   0.20944082  0.72837588]
[ 0.061989    0.20685978  0.73115122]
[ 0.06203548  0.20658778  0.73137674]
[ 0.06246063

#### Précisions sur l'exactitude globale

La valeur de l'exactitude renvoyée ci-dessus est en fait équivalente à une mesure de sensibilité, car elle mesure le nombre de patients classés dans le bon groupe clinique (équivalent des vrais positifs). On ne peut en revanche pas faire état du nombre de "vrais négatifs" car un test qui est négatif pour une des classes est en fait négatif pour 3 des 4 classes restantes... Donc même si un individu est un "vrai négatif" pour une classe donnée, il n'y a qu'une chance sur 4 pour qu'il soit en réalité dans la bonne classe... On ne peut le compter juste que s'il est lui-même bien classé. 

Ce résultat n'est donc pas à interpréter de la même façon que la classification binaire. Ici, on dépasse la notion de "vrai/faux positif/négatif", étant donné qu'il y a 5 classes. Ainsi, un classifieur aléatoire obtiendrait une exactitude de 20% et non 50%. Il est intéressant de garder cela à l'esprit en interprétant les résultats, car il est nécessairement plus difficile d'obtenir une bonne valeur d'exactitude en multi-classes qu'en classification binaire. 

#### Résultats par classes individuelles 

Ci-dessous sont représentés les valeurs de sensibilité de chaque classe. Il s'agit là encore des "vrais positifs", c'est à dire des individus placés dans le bon groupe clinique. 

Cela ne rend donc pas compte des "faux positifs". Ainsi, si on imagine que les individus soient tous mis en classe 1. L'exactitude de la classe 1 serait alors de 100%, car tous les éléments de la classe 1 ont été bien classés. Cependant, un tel test serait complètement inefficace pour déterminer la classe 1, car le nombre de faux positifs relatifs à cette classe serait énorme... 

Les résultats présentés ici ne servent donc qu'à quantifier le nombre d'éléments bien classés, pour voir quelles classes sont les mieux reconnues, mais ils ne doivent pas être interprétés comme un évaluateur de la qualité de la classification globale pour une classe donnée. Ce résultat sera donné plus loin. 

In [13]:
sens_0 = 0
sens_1 = 0
sens_2 = 0
sens_3 = 0
sens_4 = 0

print "Sensibilité par classe :"

for j in range (41) :
    sens_0 = sens_0 + accuracy[j]

print "Classe 0 = %2.2f%%" % (100*sens_0/41)

for k in range (41, 47) :
    sens_1 = sens_1 + accuracy[k]

print "Classe 1 = %2.2f%%" % (100*sens_1/6)

for l in range (47, 76) :
    sens_2 = sens_2 + accuracy[l]

print "Classe 2 = %2.2f%%" % (100*sens_2/29)

for m in range (76,106) :
    sens_3 = sens_3 + accuracy[m]

print "Classe 3 = %2.2f%%" % (100*sens_3/30)

for n in range (106,112) :
    sens_4 = sens_4 + accuracy[n]

print "Classe 4 = %2.2f%%" % (100*sens_4/6)

Sensibilité par classe :
Classe 0 = 82.93%
Classe 1 = 0.00%
Classe 2 = 72.41%
Classe 3 = 90.00%
Classe 4 = 33.33%


In [14]:
print result

[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 3.0, 2.0, 0.0, 2.0, 0.0, 2.0, 2.0, 2.0, 0.0, 0.0, 2.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 0.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 0.0, 0.0, 0.0, 4.0]


### Ratio de faux positifs par classe

Comme nous l'avons précisé, les résultats précédents ne font état que des éléments bien classés pour chacune des classes, et ne renseignent pas sur les erreurs commises (les "faux classe N") 

La cellule suivante renseigne donc sur le nombre de "faux classe N" pour chacune des classes : pour une classe N donnée, ce ratio donne le nombre d'éléments qui ont été mis par erreur dans N. 

In [15]:
sum_0 = 0
sum_1 = 0
sum_2 = 0
sum_3 = 0
sum_4 = 0

for i in range(len(Yall)) :
    if result[i] == 0 :
        sum_0 = sum_0 + 1
    elif result[i] == 1 :
        sum_1 = sum_1 + 1
    elif result[i] == 2 :
        sum_2 = sum_2 + 1
    elif result[i] == 3 :
        sum_3 = sum_3 + 1
    elif result[i] == 4 :
        sum_4 = sum_4 + 1
        
print 'Ratio de faux positifs par classe :'

print "Classe 0 = %2.2f%%" % (100*(sum_0 - sens_0)/41)
print "Classe 1 = %2.2f%%" %(100*(sum_1 - sens_1)/6)
print "Classe 2 = %2.2f%%" %(100*(sum_2 - sens_2)/29)
print "Classe 3 = %2.2f%%" %(100*(sum_3 - sens_3)/30)
print "Classe 4 = %2.2f%%" %(100*(sum_4 - sens_4)/6)

Ratio de faux positifs par classe :
Classe 0 = 36.59%
Classe 1 = 16.67%
Classe 2 = 31.03%
Classe 3 = 10.00%
Classe 4 = 0.00%


### Exactitude de la prédiction

Ultimement, nous voudrions savoir avec quelle probabilité un patient prédit classe N appartient véritablement à la classe N. C'est ce que fait la cellule suivante.

In [16]:
if sum_0 != 0 :
    proba_0 = sens_0/sum_0
else :
    proba_0 = 0

if sum_1 != 0 :
    proba_1 = sens_1/sum_1
else :
    proba_1 = 0

if sum_2 != 0 :
    proba_2 = sens_2/sum_2
else :
    proba_2 = 0

if sum_3 != 0 :
    proba_3 = sens_3/sum_3
else :
    proba_3 = 0

if sum_4 != 0 :
    proba_4 = sens_4/sum_4
else :
    proba_4 = 0


print 'Exactitude par classe :'
print "Classe 0 = %2.2f%%" % (100*proba_0)
print "Classe 1 = %2.2f%%" % (100*proba_1)
print "Classe 2 = %2.2f%%" % (100*proba_2)
print "Classe 3 = %2.2f%%" % (100*proba_3)
print "Classe 4 = %2.2f%%" % (100*proba_4)

Exactitude par classe :
Classe 0 = 69.39%
Classe 1 = 0.00%
Classe 2 = 70.00%
Classe 3 = 90.00%
Classe 4 = 100.00%


In [17]:
print "Temps écoulé = " 
print toc-tic

Temps écoulé = 
455.188217878
